In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv("training_data.csv", names=["text","intent"])
test_df = pd.read_csv("testing_data.csv", names=["text","intent"])
val_df = pd.read_csv("validation_data.csv", names=["text","intent"])
train_df.head()

,text,intent
NaN,0,1
0.0,what expression would i use to say i love you ...,translate
1.0,can you tell me how to say 'i do not speak muc...,translate
2.0,"what is the equivalent of, 'life is good' in f...",translate
3.0,"tell me how to say, 'it is a beautiful morning...",translate


In [3]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
# Shuffle the training DataFrame
train_df_shuffled = train_df.sample(frac=1).reset_index(drop=True)

In [4]:
# Tokenize text data
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_df_shuffled['text'])

In [5]:
# Tokenize training text data
train_sequences = tokenizer.texts_to_sequences(train_df_shuffled['text'])

# Tokenize validation text data
val_sequences = tokenizer.texts_to_sequences(val_df['text'])

# Tokenize testing text data
test_sequences = tokenizer.texts_to_sequences(test_df['text'])

In [13]:
train_sequences[0]

[309, 4, 442, 361, 7, 301, 1083, 43, 9]

In [14]:
train_df_shuffled['text'][0]

'share the nutrition info for french fries with me'

In [11]:
# Pad sequences to ensure uniform length
max_length = max(len(seq) for seq in train_sequences + val_sequences + test_sequences)
train_padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length, padding='post')
val_padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(val_sequences, maxlen=max_length, padding='post')
test_padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length, padding='post')

In [12]:
train_padded_sequences[0]

array([ 309,    4,  442,  361,    7,  301, 1083,   43,    9,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])

In [15]:
# Convert intents to unique integer labels
intent_label_mapping = {label: idx for idx, label in enumerate(train_df_shuffled['intent'].unique())}
train_df_shuffled['intent_labels'] = train_df_shuffled['intent'].map(intent_label_mapping)
val_df['intent_labels'] = val_df['intent'].map(intent_label_mapping)
test_df['intent_labels'] = test_df['intent'].map(intent_label_mapping)

In [17]:
train_df_shuffled.sample()

,text,intent,intent_labels
1676,lets go threw some trivia on sports,fun_fact,48


In [19]:
# Convert integer labels to one-hot encoding
num_intents = len(intent_label_mapping)
train_intent_labels = tf.keras.utils.to_categorical(train_df_shuffled['intent_labels'], num_classes=num_intents)
val_intent_labels = tf.keras.utils.to_categorical(val_df['intent_labels'], num_classes=num_intents)
test_intent_labels = tf.keras.utils.to_categorical(test_df['intent_labels'], num_classes=num_intents)

In [22]:
train_intent_labels[0], len(train_intent_labels)

(array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32),
 15001)

In [24]:
# Define RNN model with additional layers and increased units in the embedding layer
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=200, input_length=max_length),
    tf.keras.layers.LSTM(128, return_sequences=True),  # Adding another LSTM layer with return_sequences=True
    tf.keras.layers.LSTM(64),  # Adding another LSTM layer
    tf.keras.layers.Dense(128, activation='relu'),  # Adding a Dense hidden layer
    tf.keras.layers.Dense(num_intents, activation='softmax')  # Output layer with softmax activation
])

In [25]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
# Train the model with validation data
model.fit(train_padded_sequences, train_intent_labels, epochs=50, batch_size=32, validation_data=(val_padded_sequences, val_intent_labels))

Epoch 1/50
469/469 [==============================] - 31s 54ms/step - loss: 4.5527 - accuracy: 0.0213 - val_loss: 3.8022 - val_accuracy: 0.0580
Epoch 2/50
469/469 [==============================] - 28s 60ms/step - loss: 3.1555 - accuracy: 0.1147 - val_loss: 2.7402 - val_accuracy: 0.2079
Epoch 3/50
469/469 [==============================] - 26s 55ms/step - loss: 2.1805 - accuracy: 0.3037 - val_loss: 2.1629 - val_accuracy: 0.3742
Epoch 4/50
469/469 [==============================] - 27s 58ms/step - loss: 1.4296 - accuracy: 0.5425 - val_loss: 1.6861 - val_accuracy: 0.5698
Epoch 5/50
469/469 [==============================] - 27s 57ms/step - loss: 0.9383 - accuracy: 0.7128 - val_loss: 1.5025 - val_accuracy: 0.6538
Epoch 6/50
469/469 [==============================] - 25s 54ms/step - loss: 0.6213 - accuracy: 0.8279 - val_loss: 1.4401 - val_accuracy: 0.7128
Epoch 7/50
469/469 [==============================] - 26s 55ms/step - loss: 0.4522 - accuracy: 0.8768 - val_loss: 1.4231 - val_accuracy:

In [27]:
model.evaluate(test_padded_sequences, test_intent_labels)

141/141 [==============================] - 2s 11ms/step - loss: 1.4283 - accuracy: 0.8574


[1.428333044052124, 0.8573650121688843]

In [28]:
# Test Accuracy= 85.73%
# Training Data Accuracy= 99.59%